# Asynch Activity

##### 1. Importing Libraries:

In [1]:
import pandas as pd
import math
import sqlite3
import json
from sklearn.preprocessing import StandardScaler
import numpy as np

##### 2. Loading Data:

In [2]:
data=pd.read_csv('Social_Network_Ads_practice.csv')

##### 3. Handling Missing Data:

In [3]:
age= math.floor(data["Age"].median())
data["Age"]=data["Age"].fillna(age)

##### 4. Grouping Data by Gender and Age:

In [4]:
gender={i:[] for i in data["Gender"]}
for Gender,age in zip(data["Gender"],data["Age"]):
    gender[Gender]+=[age]

##### 5. Standardizing Data:

In [5]:
# feature Scaling
for i in gender:
    scaler = StandardScaler()
    gender[i] = scaler.fit_transform(np.array(gender[i]).reshape(-1,1))

    x = []
    for j in gender[i]:
        for k in j:
            x.append(k)
    gender[i] = x
    

##### 6. Storing Data in a SQLite Database:

In [9]:
for i in gender:
    # Connect to the database(or create it if it doesn't exist)
    conn = sqlite3.connect('Asynch_Activtiy_db.db')

    # Create a cursor object to interact with the database
    cursor = conn.cursor()

    # Create a table if it doesn't exist
    cursor.execute('''CREATE TABLE IF NOT EXISTS users(id INTEGER PRIMARY KEY)''')

    try:
        # Add a new column named 'email' to the 'users' table
        cursor.execute('ALTER TABLE users ADD COLUMN ' + i + ' TEXT')
    except sqlite3.OperationalError:
        pass

    # Convert the list to a JSON string
    gender_json = json.dumps(gender[i])

    # Insert data into the table
    cursor.execute("INSERT INTO users (" + i + ") VALUES(?)", (gender_json,))

    # Commit the changes
    conn.commit()

# Close the cursor and connection when done
cursor.close()
conn.close()